In [1]:
import numpy as np
import os
from scipy.linalg import eigh, qr, null_space
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy.sparse import kron, identity, csr_matrix, csc_matrix, lil_matrix, dok_matrix, issparse, coo_matrix
from scipy.sparse.linalg import eigsh, eigs, LinearOperator, ArpackNoConvergence
from scipy.optimize import curve_fit
from qutip import Qobj, ptrace, entropy_vn, qeye, tensor
from tqdm import tqdm   
import time
from itertools import product
from functools import reduce
import torch
import torch.optim as optim
from torch.autograd import Variable
import sympy as sp
from collections import Counter

In [2]:
class CountedOp(LinearOperator):
    def __init__(self, A):
        self.A = A
        self.shape = A.shape
        self.dtype = getattr(A, 'dtype', np.float64)
        self.calls = 0
    def _matvec(self, x):
        self.calls += 1
        return self.A @ x

In [3]:
def pauli_x():
    """Pauli X matrix."""
    return np.array([[0, 1], [1, 0]])

def pauli_z():
    """Pauli Z matrix."""
    return np.array([[1, 0], [0, -1]])

def dodecahedral_bonds(): #20 vertices
    """
    Defines the connectivity of a true 20-vertex dodecahedral molecular structure.

    Returns:
        list of tuples: Each tuple (i, j) represents a bond between spin i and spin j.
    """
    bonds = [
    (0, 13), (0, 14), (0, 15),
    (1, 4), (1, 5), (1, 12),
    (2, 6), (2, 13), (2, 18),
    (3, 7), (3, 14), (3, 19),
    (4, 10), (4, 18),
    (5, 11), (5, 19),
    (6, 10), (6, 15),
    (7, 11), (7, 15),
    (8, 9), (8, 13), (8, 16),
    (9, 14), (9, 17),
    (10, 11),
    (12, 16), (12, 17),
    (16, 18),
    (17, 19)
]

    return bonds


def transverse_field_ising_dodecahedral(N, J, h):
    """
    Constructs the Hamiltonian for the transverse field Ising model on a dodecahedral molecular structure.

    Parameters:
        N (int): Number of spins (should match the dodecahedral molecule, typically N=12).
        J (float): Interaction strength.
        h (float): Transverse field strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 20:
        raise ValueError("Dodecahedral molecules typically have N = 20 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    X = csr_matrix(pauli_x())
    Z = csr_matrix(pauli_z())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)

    # Get dodecahedral bonds
    bonds = dodecahedral_bonds()

    # Interaction term: J * sigma_i^x * sigma_j^x for dodecahedral connectivity
    for i, j in bonds:
        term = 1
        for k in range(N):
            if k == i or k == j:
                term = kron(term, X, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += J * term
    
    # Transverse field term: -h * sigma_i^z
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = kron(term, Z, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += -h * term
    
    return H

def ising_dodecahedron(N, J):
    """
    Constructs the Hamiltonian for the transverse field Ising model on a dodecahedral molecular structure without transverse field.

    Parameters:
        N (int): Number of spins (should match the dodecahedral molecule, typically N=20).
        J (float): Interaction strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 20:
        raise ValueError("Dodecahedral molecules typically have N = 20 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    X = csr_matrix(pauli_x())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)

    # Get Dodecahedron bonds
    bonds = dodecahedral_bonds()

    # Interaction term: J * sigma_i^x * sigma_j^x for Dodecahedron connectivity
    for i, j in bonds:
        term = 1
        for k in range(N):
            if k == i or k == j:
                term = kron(term, X, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += J * term
    
    return H

def transverse_field_dodecahedral(N, h):
    """
    Constructs the Hamiltonian for the transverse field Ising model on a dodecahedral molecular structure.

    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        h (float): Transverse field strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 20:
        raise ValueError("Dodecahedral molecules typically have N = 20 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    Z = csr_matrix(pauli_z())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)

    # Get dodecahedral bonds
    bonds = dodecahedral_bonds()

    # Transverse field term: -h * sigma_i^z
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = kron(term, Z, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += -h * term
    
    return H

def H_shifted(H0, x):
    H_s = H0 - x * identity(H0.shape[0], format="csr")
    H_s = H_s.tocsr()
    return H_s


def H_shifted_sq(H0, x):
    H_s = H0 - x * identity(H0.shape[0], format="csr")
    H_s_sq = H_s @ H_s
    H_s_sq = H_s_sq.tocsr()
    return H_s_sq

def resid_norms(Aop, evals, evecs):
    # Works with either a matrix or LinearOperator
    R = Aop @ evecs - evecs * evals
    return np.linalg.norm(R, axis=0)

def combine_cols(V):
    # Make a v0 from current Ritz vectors (works even if we have just 1)
    c = np.random.randn(V.shape[1])
    c /= np.linalg.norm(c)
    v0 = V @ c
    v0 /= np.linalg.norm(v0)
    return v0

def probe_eigsh(
    A, k=6, which='LM',  # 'SA'/'LA' for extremal; with sigma use which='LM'
    maxiter_step=10, steps=10,
    ncv=None, tol=1e-10, v0=None,
    sigma=18.1, OPinv=None, mode='normal',
    verbose=True
):
    """
    Run eigsh in short bursts, printing residuals after each burst.
    Works with normal and shift-invert (set sigma and optionally OPinv).
    """
    last_v0 = v0
    converged_wV = None

    for s in range(1, steps+1):
        try:
            w, V = eigsh(A, k=k, which=which, ncv=ncv, tol=tol,
                         maxiter=maxiter_step, v0=last_v0,
                         sigma=sigma, OPinv=OPinv, mode=mode,
                         return_eigenvectors=True)
            if verbose:
                res = resid_norms(A if sigma is None else LinearOperator(A.shape, matvec=lambda x: A @ x), w, V)
                print(f"[step {s}] CONVERGED: k={k}  maxiter={maxiter_step}  "
                      f"residuals min/median/max = {res.min():.2e}/{np.median(res):.2e}/{res.max():.2e}", flush=True)
            converged_wV = (w, V)
            break
        except ArpackNoConvergence as err:
            w = err.eigenvalues
            V = err.eigenvectors
            if V is not None and V.size:
                res = resid_norms(A if sigma is None else LinearOperator(A.shape, matvec=lambda x: A @ x), w, V)
                if verbose:
                    print(f"[step {s}] PARTIAL: {V.shape[1]} Ritz vectors; "
                          f"residuals min/median/max = {res.min():.2e}/{np.median(res):.2e}/{res.max():.2e}", flush=True)
                last_v0 = combine_cols(V)  # warm-start next burst
                converged_wV = (w, V)
            else:
                if verbose:
                    print(f"[step {s}] NO RITZ VECTORS YET — try larger ncv or more steps.", flush=True)
                # random warm start to shake things up a bit
                n = A.shape[0]
                last_v0 = np.random.randn(n); last_v0 /= np.linalg.norm(last_v0)

    return converged_wV  # (w, V) from last attempt (maybe partial)



In [4]:
N = 20  # Number of spins
J = 1.0  # Interaction strength
h = 3.0  # Transverse field strength # this is the value in the paper. maybe try other values too, including the critical value one (h=J=1)


# Assuming transverse_field_ising is defined and returns a sparse Hermitian matrix
H = transverse_field_ising_dodecahedral(N, J, h)
Hi = ising_dodecahedron(N, J)
Htf = transverse_field_dodecahedral(N, h)

In [8]:
Aop = CountedOp(H)
try:
    w, V = eigsh(Aop, k=6, sigma=-18.1, which='LM', maxiter=1, ncv=48, tol=1e-8)
except ArpackNoConvergence:
    pass
print("matvec calls:", Aop.calls, flush=True)

KeyboardInterrupt: 

In [12]:
eivt, eigt = eigsh(H, k=10, which='SA', maxiter=5, ncv=64, tol=1e-8)

In [10]:
print(eivt)

[-62.51489576 -58.59394385 -58.59394385 -58.59394385 -58.25359416
 -58.25359416 -58.25359416 -56.05370058 -56.05370058 -55.1979083 ]


In [ ]:
E0 = -18.0
H_shifted = H - E0 * identity(H.shape[0], format="csr")
#H_sq = H_shifted @ H_shifted
eivt, eigt = eigsh(H_shifted, k=1, which='SM', maxiter=300, ncv=256, tol=1e-8)
# The closest eigenvalue of H is: E0 ± sqrt(eivt[0])

KeyboardInterrupt: 